# Face Greeter: Real-Time Video Analytics System

This Jupyter Notebook powers the backend for FaceGreeter, a real-time video analytics system for businesses. This simplified version tests Flask and ngrok stability to debug kernel crashes and ngrok tunnel issues. The full system uses `face_recognition` for face detection, OpenCV for video processing, and a Flask API to manage camera connections, video streaming, and customer insights via the xAI Grok API. It integrates with the Vercel frontend at https://facegreeter.vercel.app.

## Prerequisites
- Google Colab environment with internet access.
- `NGROK_AUTHTOKEN` stored in Colab Secrets.
- GitHub repository `ai-greeter-core` at https://github.com/ATEMMETA/ai-greeter-core (public).

## Steps
1. Clear existing directory and clone ai-greeter-core or create fallback files.
2. Install dependencies from requirements.txt.
3. Set up environment variables.
4. Run simplified Flask API with ngrok.
5. Test stability and monitor resources.

**Security Note**: Securely handle `NGROK_AUTHTOKEN` in Colab Secrets.

In [ ]:
# Clear directory, clone repository or create fallback files, install dependencies
import os
import shutil
from google.colab import userdata
import subprocess
import psutil

os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'  # Disable frozen modules warning

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Clear existing ai-greeter-core directory
if os.path.exists('/content/ai-greeter-core'):
    shutil.rmtree('/content/ai-greeter-core')

# Create ai-greeter-core directory
os.makedirs('/content/ai-greeter-core', exist_ok=True)

# Attempt to clone ai-greeter-core repository
try:
    github_token = userdata.get('GITHUB_TOKEN')
except userdata.SecretNotFoundError:
    github_token = None
    print('GITHUB_TOKEN not found in Colab Secrets. Using public repository.')

repo_url = 'https://github.com/ATEMMETA/ai-greeter-core.git'
if github_token:
    clone_url = f'https://{github_token}@github.com/ATEMMETA/ai-greeter-core.git'
    clone_command = f'git clone {clone_url} /content/ai-greeter-core'
else:
    clone_command = f'git clone {repo_url} /content/ai-greeter-core'

# Run git clone with verbose output
print(f'Cloning repository: {clone_command}')
clone_process = subprocess.run(clone_command, shell=True, capture_output=True, text=True)
if clone_process.returncode != 0 or not os.path.exists('/content/ai-greeter-core/requirements.txt'):
    print(f'Git clone failed or requirements.txt missing. Error: {clone_process.stderr}')
    print('Creating fallback files...')
    # Create requirements.txt
    with open('/content/ai-greeter-core/requirements.txt', 'w') as f:
        f.write('flask==3.0.3\n')
        f.write('flask-cors==5.0.0\n')
        f.write('pyngrok==7.1.6\n')
        f.write('python-dotenv==1.0.1\n')
    # Create combined_app.py
    with open('/content/ai-greeter-core/combined_app.py', 'w') as f:
        f.write('''from flask import Flask, jsonify\n''')
        f.write('from flask_cors import CORS\n')
        f.write('import os\n')
        f.write('import logging\n\n')
        f.write('logging.basicConfig(level=logging.INFO, format=\'%(asctime)s - %(levelname)s - %(message)s\')\n')
        f.write('logger = logging.getLogger(__name__)\n\n')
        f.write('app = Flask(__name__)\n')
        f.write('CORS(app, resources={r"/*": {"origins": "https://facegreeter.vercel.app"}})\n\n')
        f.write('@app.route(\'/hello\', methods=[\'GET\'])\n')
        f.write('def hello():\n')
        f.write('    logger.info("Received request to /hello")\n')
        f.write('    return jsonify({"success": True, "message": "Hello from Flask!"})\n\n')
        f.write('if __name__ == \'__main__\':\n')
        f.write('    app.run(host=\'0.0.0.0\', port=8000)\n')
else:
    print('Git clone succeeded.')

# Purge pip cache
!pip cache purge

# Install Python dependencies with force reinstall
!pip install --no-cache-dir --force-reinstall -r /content/ai-greeter-core/requirements.txt

# Verify installed packages
!pip list | grep -E 'flask|pyngrok|python-dotenv'
print('Dependencies installed. Please go to \'Runtime\' > \'Restart runtime\' and then re-run all cells.')
print('Monitor RAM/CPU usage in Colab\'s top-right corner during the next steps.')

# Log resource usage after install
print(f'Post-install RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-install CPU: {psutil.cpu_percent()}%')

In [ ]:
# Set up environment variables
from google.colab import userdata
import os

# Check for required secrets
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')

# Ensure the directory exists
os.makedirs('/content/ai-greeter-core', exist_ok=True)

# Write .env file
with open('/content/ai-greeter-core/.env', 'w') as f:
    f.write(f'NGROK_AUTHTOKEN={ngrok_token}\n')
    f.write('VERCEL_URL=https://facegreeter.vercel.app\n')
    f.write('VERCEL_API_BASE_URL=https://facegreeter.vercel.app\n')

In [ ]:
# Run Flask with ngrok
from pyngrok import ngrok
import os
import subprocess
import time
from dotenv import load_dotenv
import psutil

# Kill existing ngrok and Flask processes
os.system('pkill ngrok')
os.system('pkill python')
os.system('fuser -k 8000/tcp')

# Load environment variables
load_dotenv('/content/ai-greeter-core/.env')
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in .env file.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Log resource usage before Flask
print(f'Pre-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Pre-Flask CPU: {psutil.cpu_percent()}%')

# Start Flask server with error logging
print('Starting Flask server...')
flask_process = subprocess.Popen(
    ['python', '/content/ai-greeter-core/combined_app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for Flask to start
time.sleep(5)

# Check if Flask is running
if flask_process.poll() is not None:
    stdout, stderr = flask_process.communicate()
    print(f'Flask failed to start. Error: {stderr}')
    raise RuntimeError('Flask server crashed. Check error above.')

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    flask_process.terminate()
    raise

print(f'Flask server started with PID: {flask_process.pid}')

# Debug: Check running processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

# Keep the notebook running
try:
    input('Press Enter to stop the Flask server...')
finally:
    flask_process.terminate()
    os.system('pkill ngrok')
    os.system('pkill python')
    os.system('fuser -k 8000/tcp')